In [28]:

from bs4 import BeautifulSoup
import requests

In [29]:

import urllib.parse as urlparse
url = "https://www.timeanddate.com/weather/uk/london/historic?month=&year="

# set range of months and year
month=range(1,12)
year=[2021]

# add month and years to url
def insertChar(mystring, mont,year):
    mystring   =  mystring[:61] + mont + mystring[61:67]+ year + mystring[67:]
    return mystring

result_url=[]
for i in month:
  for j in year:
    result_url.append(insertChar(url, str(i),str(j)))

# print all urls
result_url


['https://www.timeanddate.com/weather/uk/london/historic?month=1&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=2&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=3&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=4&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=5&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=6&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=7&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=8&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=9&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=10&year=2021',
 'https://www.timeanddate.com/weather/uk/london/historic?month=11&year=2021']

In [30]:
import os

# get content of web for each date
for idx,url in enumerate(result_url):
  months_weather = requests.get(str(url))
  soup=BeautifulSoup(months_weather.content, 'html.parser')
  #print(months_weather)

  # Create the directory if it doesn't exist
  directory = os.path.dirname("/path/{}soupFile.txt".format(idx))
  if not os.path.exists(directory):
      os.makedirs(directory)

  #save them into the text file
  with open("/path/{}soupFile.txt".format(idx), "w") as f:
        f.write(str(soup))

In [31]:

# for read soupfiles- it's just a test :)
for idx in range(0,12):
  f = open("/path/{}soupFile.txt".format(idx))
  soup = f.read()
  bs = BeautifulSoup(soup,"html.parser")

In [32]:
import pandas as pd
import json
dfs = pd.DataFrame()
for idx in range(0,12):
  ddf = pd.DataFrame(columns=['date', 'date_id','wind', 'desc','temp','baro','hum'])
  f = open("/path/{}soupFile.txt".format(idx))
  r = f.read()
  soup = BeautifulSoup(r,"html.parser")
  scripts = soup.find_all('script')
  jsonObj = None
  for script in scripts:
      if 'var data='  in script.text:
          jsonStr = script.text.strip()

          jsonStr = jsonStr.split('var data=')[1]
          jsonStr = jsonStr.split(';')[0]

          jsonObj = json.loads(jsonStr)
  # Create a list to store data for this iteration
  data_to_append = []
  for item in jsonObj['detail']:
    # Check if the 'temp' key exists before accessing it
    date = item['ds']
    date_id=item['date']
    wind = item['wind']
    desc=item['desc']
    temp = item.get('temp', None)  # Use get() to avoid KeyError if 'temp' is missing
    baro = item.get('baro', None)  # Use get() to avoid KeyError if 'baro' is missing
    hum = item.get('hum', None)  # Use get() to avoid KeyError if 'hum' is missing
    # Append data as a dictionary to the list
    data_to_append.append({'date': date, 'date_id': date_id, 'wind':wind, 'desc': desc, 'temp': temp, 'baro': baro, 'hum': hum })
  # Create a temporary DataFrame and concatenate
  temp_df = pd.DataFrame(data_to_append)
  ddf = pd.concat([ddf, temp_df], ignore_index=True)

  dfs = pd.concat([dfs, ddf], axis=0).reset_index(drop=True)

# print
dfs.head(5)

<ipython-input-32-47e103e3ca48>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ddf = pd.concat([ddf, temp_df], ignore_index=True)
<ipython-input-32-47e103e3ca48>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ddf = pd.concat([ddf, temp_df], ignore_index=True)
<ipython-input-32-47e103e3ca48>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the resul

,date,date_id,wind,desc,temp,baro,hum
0,"Friday, January 1, 2021, 12:00 am — 6:00 am",1.609459e+12,4.971,Low clouds.,32.0,29.82,99
1,"Friday, January 1, 2021, 6:00 am — 12:00 pm",1.609481e+12,4.971,Low clouds.,33.8,29.86,94
2,"Friday, January 1, 2021, 12:00 pm — 6:00 pm",1.609502e+12,4.350,Drizzle. Low clouds.,37.4,29.87,96
3,"Friday, January 1, 2021, 6:00 pm — 12:00 am",1.609524e+12,4.971,Low clouds.,39.2,29.93,96
4,"Saturday, January 2, 2021, 12:00 am — 6:00 am",1.609546e+12,7.457,Low clouds.,39.2,29.97,92


In [33]:
# way with pd.read_html
for idx in range(0,12):
  f = open("/path/{}soupFile.txt".format(idx))
  r = f.read()
  soup = BeautifulSoup(r,"html.parser")
  dfss = pd.DataFrame()
  table = soup.find('table', id='wt-his')
  dff = pd.read_html(str(table))[0]
  # Dropping a level down
  dff.columns = dff.columns.droplevel(0)
  dff=dff.drop(['Unnamed: 1_level_1','Unnamed: 5_level_1','Visibility'], axis = 1)
  dfss = pd.concat([dfss, dff], axis=0).reset_index(drop=True)

dfss.head(5)

<ipython-input-33-7bddb94ede18>:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dff = pd.read_html(str(table))[0]
<ipython-input-33-7bddb94ede18>:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dff = pd.read_html(str(table))[0]
<ipython-input-33-7bddb94ede18>:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dff = pd.read_html(str(table))[0]
<ipython-input-33-7bddb94ede18>:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dff = pd.read_html(str(table))[0]
<ipython-input-33-7bddb94ede18>:8: Futur

,Time,Temp,Weather,Wind,Humidity,Barometer
0,"12:50 am Wed, Dec 1",52 °F,Passing clouds.,16 mph,88%,"29.42 ""Hg"
1,1:50 am,52 °F,Partly cloudy.,16 mph,88%,"29.39 ""Hg"
2,2:50 am,52 °F,Passing clouds.,17 mph,82%,"29.36 ""Hg"
3,3:50 am,50 °F,Partly cloudy.,15 mph,82%,"29.36 ""Hg"
4,4:50 am,50 °F,Partly cloudy.,15 mph,82%,"29.39 ""Hg"


In [34]:
from google.colab import files

# Save DataFrames to CSV

dfs.to_csv('Weather_london.csv', sep=',', index=False)

# Download the CSV files

files.download('Weather_london.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>